In [5]:
from match_em import analysis

results = analysis.compute_mistakes('frå neste veke av vart altså', 'fra neste veka var altså')
test_lines = [l.strip() for l in results['final_print'].split('\n') if l]

truth_lines = [l.strip() for l in '''--- UNK_ID (WER: 60.0, compounds created: 1, compounds broken up: 0)---
  f | r | å    ||  neste  ||   v | e | k | e |   | a | v    ||   v | a | r | t    ||  altså  || 
  f | r | a    ||  neste  ||   v | e | k |   |   | a |      ||   v | a | r |      ||  altså  || 
    |   | S    ||         ||     |   |   | D | D |   | D    ||     |   |   | D    ||         || 
    '''.split('\n') if l.strip() != '']

In [3]:
for l in range(len(test_lines)):
    if test_lines[l] != truth_lines[l]:
        print('''
        ---
        -{}-
        -{}-
        ---

        '''.format(truth_lines[l], test_lines[l]))

In [4]:
test_lines == truth_lines

True

In [8]:
results = analysis.compute_mistakes('frå neste veke av vart altså', 'fra neste veka var altså', distance_method='weighted_manual')
print(results['final_print'])

--- UNK_ID (WER: 66.67, compounds created: 0, compounds broken up: 0)---
  f | r | å    ||  neste  ||   v | e | k | e    ||   a | v    ||   v | a | r | t    ||  altså  || 
  f | r | a    ||  neste  ||   v | e | k | a    ||     |      ||   v | a | r |      ||  altså  || 
    |   | S    ||         ||     |   |   | S    ||   D | D    ||     |   |   | D    ||         || 




In [3]:
from match_em.distances import distance

dist = distance(
    'sau hadde dei hatt på heimgarden og sau ville han halda på med vidare',
    'sau hadde de hatt på heimgaren og sau villand holda på med videre'
)

print(dist.get_levenshtein_editops())
print()
print(dist.get_weighted_editops())

[('replace', 2, 2), ('replace', 5, 5), ('delete', 8, 8), ('replace', 9, 8), ('replace', 10, 9), ('replace', 13, 12)]

[('sub', 2, 2), ('sub', 5, 5), ('sub', 8, 8), ('delete', 9, 8), ('sub', 10, 9), ('sub', 13, 12)]


[('sub', 2, 2), ('sub', 5, 5), ('delete', 8, 8), ('sub', 9, 8),   ('sub', 10, 9), ('sub', 13, 12)]

[('sub', 2, 2), ('sub', 5, 5),  ('sub', 8, 8),  ('delete', 8, 9), ('sub', 9, 10), ('sub', 12, 13)]



sau |  hadde |  dei |  hatt |  på |  heimgarden |  og |  sau |   ville  |    han   |  halda |  på |  med |  vidare

sau |  hadde |  de  |  hatt |  på |  heimgaren  |  og |  sau |          |  villand |  holda |  på |  med |  videre
 0  |    1   |   2  |   3   |  4  |      5      |  6  |   7  |    8     |     9    |    10  |  11 |  12  |   13   
             | sub  |             |     sub     |     |      |    del   |     sub  |   sub  |     |      |   sub

sau |  hadde |  de  |  hatt |  på |  heimgaren  |  og |  sau |  villand |          |  holda |  på |  med |  videre
 0  |    1   |   2  |   3   |  4  |      5      |  6  |   7  |     8    |     9    |    10  |  11 |  12  |   13   
             | sub  |             |     sub     |     |      |    sub   |    del   |   sub  |     |      |   sub 

 
sau |  hadde |  de  |  hatt |  på |  heimgaren  |  og |  sau |  villand | holda |  på |  med |  videre
 0  |    1   |   2  |   3   |  4  |      5      |  6  |   7  |     8    |  9    |  10 |  11  |    12   

In [4]:
from copy import deepcopy
dist.compute_weighted_alignment()
candidate_paths = dist.backtrace_alignment_paths()
print(list(reversed(deepcopy(candidate_paths[0]))))
print(dist.create_editops(candidate_paths[0]))

[0, (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14)]
[('sub', 2, 2), ('sub', 5, 5), ('sub', 8, 8), ('delete', 9, 8), ('sub', 10, 9), ('sub', 13, 12)]


In [5]:
from match_em.distances import distance
from match_em.alignments import get_alignment_words, print_alignment_words
from copy import deepcopy

ref = 'frå neste veke av vart altså'
hyp = 'fra neste veka var altså'

dist3 = distance(ref, hyp)

dist3.compute_weighted_alignment()
candidate_paths = dist3.backtrace_alignment_paths()

print([(op if op[0] != 'replace' else ('sub', op[1], op[2])) for op in dist3.get_levenshtein_editops() ])
ct_1, ic_1, ref_1, hyp_1 = get_alignment_words(ref, hyp, dist3.get_levenshtein_editops())
print(ic_1)
word_align = print_alignment_words(ref_1, hyp_1, index_changes=ic_1, only_print_subs=False, do_print=True, print_char_alignments=False)

print()

print(list(reversed(deepcopy(candidate_paths[0]))))
print(dist3.create_editops(candidate_paths[0]))
ct_2, ic_2, ref_2, hyp_2 = get_alignment_words(ref, hyp, dist3.create_editops(candidate_paths[0]))
print(ic_2)
word_align = print_alignment_words(ref_2, hyp_2, index_changes=ic_2, only_print_subs=False, do_print=True, print_char_alignments=False)


[('sub', 0, 0), ('delete', 2, 2), ('sub', 3, 2), ('sub', 4, 3)]
{0: 'S', 2: ' ', 3: 'S', 4: 'S'}
 frå  ||  neste  ||  veke  ||   av   ||  vart  ||  altså  || 
 fra  ||  neste  ||        ||  veka  ||  var   ||  altså  || 
  S   ||         ||   D    ||   S    ||   S    ||         || 

[0, (0, 0), (1, 1), (2, 2), (3, 3), (3, 4), (4, 5), (5, 6)]
[('sub', 0, 0), ('sub', 2, 2), ('delete', 3, 2), ('sub', 4, 3)]
{0: 'S', 2: 'S', 3: ' ', 4: 'S'}
 frå  ||  neste  ||  veke  ||  av  ||  vart  ||  altså  || 
 fra  ||  neste  ||  veka  ||      ||  var   ||  altså  || 
  S   ||         ||   S    ||  D   ||   S    ||         || 


In [6]:
from match_em.distances import distance
from match_em.alignments import get_alignment_words, print_alignment_words
from copy import deepcopy

ref = 'sau hadde dei hatt på heimgarden og sau ville han halda på med vidare'
hyp = 'sau hadde de hatt på heimgaren og sau villand holda på med videre'

dist3 = distance(ref, hyp)

dist3.compute_weighted_alignment()
candidate_paths = dist3.backtrace_alignment_paths()

print([(op if op[0] != 'replace' else ('sub', op[1], op[2])) for op in dist3.get_levenshtein_editops() ])
ct_1, ic_1, ref_1, hyp_1 = get_alignment_words(ref, hyp, dist3.get_levenshtein_editops())
print(ic_1)
word_align = print_alignment_words(ref_1, hyp_1, index_changes=ic_1, only_print_subs=False, do_print=True, print_char_alignments=True, max_width=150)

print()

print(list(reversed(deepcopy(candidate_paths[0]))))
print(dist3.create_editops(candidate_paths[0]))
ct_2, ic_2, ref_2, hyp_2 = get_alignment_words(ref, hyp, dist3.create_editops(candidate_paths[0]))
print(ic_2)
word_align = print_alignment_words(ref_2, hyp_2, index_changes=ic_2, only_print_subs=False, do_print=True, print_char_alignments=True, max_width=150)

[('sub', 2, 2), ('sub', 5, 5), ('delete', 8, 8), ('sub', 9, 8), ('sub', 10, 9), ('sub', 13, 12)]
{2: 'S', 5: 'S', 8: ' ', 9: 'S', 10: 'S', 13: 'S'}
 sau  ||  hadde  ||   d | e | i    ||  hatt  ||  på  ||   h | e | i | m | g | a | r | d | e | n    ||  og  ||  sau  ||   v | i | l | l | e    ||     |
 sau  ||  hadde  ||   d | e |      ||  hatt  ||  på  ||   h | e | i | m | g | a | r |   | e | n    ||  og  ||  sau  ||     |   |   |   |      ||   v |
      ||         ||     |   | D    ||        ||      ||     |   |   |   |   |   |   | D |   |      ||      ||       ||   D | D | D | D | D    ||   I |

   |   | h | a | n |      ||   h | a | l | d | a    ||  på  ||  med  ||   v | i | d | a | r | e    ||
 i | l | l | a | n | d    ||   h | o | l | d | a    ||  på  ||  med  ||   v | i | d | e | r | e    ||
 I | I | S |   |   | I    ||     | S |   |   |      ||      ||       ||     |   |   | S |   |      ||


[0, (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (9, 10

### Making sure insertions don't make things weird

In [7]:
from match_em.distances import distance
from match_em.alignments import get_alignment_words, print_alignment_words
from copy import deepcopy

hyp = 'frå neste veke av vart altså'
ref = 'fra neste veka var altså'

dist3 = distance(ref, hyp)

dist3.compute_weighted_alignment()
candidate_paths = dist3.backtrace_alignment_paths()

print([(op if op[0] != 'replace' else ('sub', op[1], op[2])) for op in dist3.get_levenshtein_editops() ])
ct_1, ic_1, ref_1, hyp_1 = get_alignment_words(ref, hyp, dist3.get_levenshtein_editops())
print(ic_1)
word_align = print_alignment_words(ref_1, hyp_1, index_changes=ic_1, only_print_subs=False, do_print=True, print_char_alignments=False)

print()

print(list(reversed(deepcopy(candidate_paths[0]))))
print(dist3.create_editops(candidate_paths[0]))
ct_2, ic_2, ref_2, hyp_2 = get_alignment_words(ref, hyp, dist3.create_editops(candidate_paths[0]))
print(ic_2)
word_align = print_alignment_words(ref_2, hyp_2, index_changes=ic_2, only_print_subs=False, do_print=True, print_char_alignments=False)


[('sub', 0, 0), ('insert', 2, 2), ('sub', 2, 3), ('sub', 3, 4)]
{0: 'S', 2: ' ', 3: 'S', 4: 'S'}
 fra  ||  neste  ||        ||  veka  ||  var   ||  altså  || 
 frå  ||  neste  ||  veke  ||   av   ||  vart  ||  altså  || 
  S   ||         ||   I    ||   S    ||   S    ||         || 

[0, (0, 0), (1, 1), (2, 2), (3, 3), (4, 3), (5, 4), (6, 5)]
[('sub', 0, 0), ('sub', 2, 2), ('insert', 2, 3), ('sub', 3, 4)]
{0: 'S', 2: 'S', 3: ' ', 4: 'S'}
 fra  ||  neste  ||  veka  ||      ||  var   ||  altså  || 
 frå  ||  neste  ||  veke  ||  av  ||  vart  ||  altså  || 
  S   ||         ||   S    ||  I   ||   S    ||         || 


In [8]:
from match_em.distances import distance
from match_em.alignments import get_alignment_words, print_alignment_words
from copy import deepcopy

hyp = 'sau hadde dei hatt på heimgarden og sau ville han halda på med vidare'
ref = 'sau hadde de hatt på heimgaren og sau villand holda på med videre'

dist3 = distance(ref, hyp)

dist3.compute_weighted_alignment()
candidate_paths = dist3.backtrace_alignment_paths()

print([(op if op[0] != 'replace' else ('sub', op[1], op[2])) for op in dist3.get_levenshtein_editops() ])
ct_1, ic_1, ref_1, hyp_1 = get_alignment_words(ref, hyp, dist3.get_levenshtein_editops())
print(ic_1)
word_align = print_alignment_words(ref_1, hyp_1, index_changes=ic_1, only_print_subs=False, do_print=True, print_char_alignments=True, max_width=150)

print()

print(list(reversed(deepcopy(candidate_paths[0]))))
print(dist3.create_editops(candidate_paths[0]))
ct_2, ic_2, ref_2, hyp_2 = get_alignment_words(ref, hyp, dist3.create_editops(candidate_paths[0]))
print(ic_2)
word_align = print_alignment_words(ref_2, hyp_2, index_changes=ic_2, only_print_subs=False, do_print=True, print_char_alignments=True, max_width=150)

[('sub', 2, 2), ('sub', 5, 5), ('insert', 8, 8), ('sub', 8, 9), ('sub', 9, 10), ('sub', 12, 13)]
{2: 'S', 5: 'S', 8: ' ', 9: 'S', 10: 'S', 13: 'S'}
 sau  ||  hadde  ||   d | e |      ||  hatt  ||  på  ||   h | e | i | m | g | a | r |   | e | n    ||  og  ||  sau  ||     |   |   |   |      ||   v |
 sau  ||  hadde  ||   d | e | i    ||  hatt  ||  på  ||   h | e | i | m | g | a | r | d | e | n    ||  og  ||  sau  ||   v | i | l | l | e    ||     |
      ||         ||     |   | I    ||        ||      ||     |   |   |   |   |   |   | I |   |      ||      ||       ||   I | I | I | I | I    ||   D |

 i | l | l | a | n | d    ||   h | o | l | d | a    ||  på  ||  med  ||   v | i | d | e | r | e    ||
   |   | h | a | n |      ||   h | a | l | d | a    ||  på  ||  med  ||   v | i | d | a | r | e    ||
 D | D | S |   |   | D    ||     | S |   |   |      ||      ||       ||     |   |   | S |   |      ||


[0, (0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 9

In [9]:
from match_em.distances import distance
from match_em.alignments import get_alignment_words, print_alignment_words
from copy import deepcopy

ref = 'gjenferd salget er rekordstort sier mona ek informasjonsansvarlig i aschehoug'
hyp = 'janferdssalget er rekordstort sier mona ek informasjonsansvarlig i askehaug'

dist3 = distance(ref, hyp)

print([(op if op[0] != 'replace' else ('sub', op[1], op[2])) for op in dist3.get_levenshtein_editops() ])
ct_1, ic_1, ref_1, hyp_1 = get_alignment_words(ref, hyp, dist3.get_levenshtein_editops())
print(ic_1)
word_align = print_alignment_words(ref_1, hyp_1, index_changes=ic_1, only_print_subs=False, do_print=True, print_char_alignments=True, max_width=150)

print()

dist3.compute_weighted_alignment()
candidate_paths = dist3.backtrace_alignment_paths()

print(list(reversed(deepcopy(candidate_paths[0]))))
print(dist3.create_editops(candidate_paths[0]))
ct_2, ic_2, ref_2, hyp_2 = get_alignment_words(ref, hyp, dist3.create_editops(candidate_paths[0]))
print(ic_2)
word_align = print_alignment_words(ref_2, hyp_2, index_changes=ic_2, only_print_subs=False, do_print=True, print_char_alignments=True, max_width=150)

[('delete', 0, 0), ('sub', 1, 0), ('sub', 9, 8)]
{0: ' ', 1: 'S', 9: 'S'}
  g | j | e | n | f | e | r | d    ||     |   |   |   |   |   |   |   | s | a | l | g | e | t    ||  er  ||  rekordstort  ||  sier  ||  mona  ||  ek  
    |   |   |   |   |   |   |      ||   j | a | n | f | e | r | d | s | s | a | l | g | e | t    ||  er  ||  rekordstort  ||  sier  ||  mona  ||  ek  
  D | D | D | D | D | D | D | D    ||   I | I | I | I | I | I | I | I |   |   |   |   |   |      ||      ||               ||        ||        ||      

||  informasjonsansvarlig  ||  i  ||   a | s | c | h | e | h | o | u | g    ||
||  informasjonsansvarlig  ||  i  ||   a | s |   | k | e | h | a | u | g    ||
||                         ||     ||     |   | D | S |   |   | S |   |      ||


[0, (0, 0), (1, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10)]
[('sub', 0, 0), ('delete', 1, 0), ('sub', 9, 8)]
{0: 'S', 1: ' ', 9: 'S'}
  g | j | e | n | f | e | r | d |   |   |   |   |   |   |      || 

In [10]:
ref = 'i følge yr dot no kommer regnet fra øst'
hyp = 'ifølge yr no kommer regnet fra øst'

from match_em.distances import distance
from match_em.alignments import get_alignment_words, print_alignment_words
from copy import deepcopy

dist3 = distance(ref, hyp)

print([(op if op[0] != 'replace' else ('sub', op[1], op[2])) for op in dist3.get_levenshtein_editops() ])
ct_1, ic_1, ref_1, hyp_1 = get_alignment_words(ref, hyp, dist3.get_levenshtein_editops())
print(ic_1)
word_align = print_alignment_words(ref_1, hyp_1, index_changes=ic_1, only_print_subs=False, do_print=True, print_char_alignments=True, max_width=150)

print()

dist3.compute_weighted_alignment()
candidate_paths = dist3.backtrace_alignment_paths()

print(list(reversed(deepcopy(candidate_paths[0]))))
print(dist3.create_editops(candidate_paths[0]))
ct_2, ic_2, ref_2, hyp_2 = get_alignment_words(ref, hyp, dist3.create_editops(candidate_paths[0]))
print(ic_2)
word_align = print_alignment_words(ref_2, hyp_2, index_changes=ic_2, only_print_subs=False, do_print=True, print_char_alignments=True, max_width=150)

[('delete', 0, 0), ('sub', 1, 0), ('delete', 3, 2)]
{0: ' ', 1: 'S', 3: ' '}
  i    ||     | f | ø | l | g | e    ||  yr  ||   d | o | t    ||  no  ||  kommer  ||  regnet  ||  fra  ||  øst  || 
       ||   i | f | ø | l | g | e    ||  yr  ||     |   |      ||  no  ||  kommer  ||  regnet  ||  fra  ||  øst  || 
  D    ||   I |   |   |   |   |      ||      ||   D | D | D    ||      ||          ||          ||       ||       || 

[0, (0, 1), (1, 2), (2, 3), (2, 4), (3, 5), (4, 6), (5, 7), (6, 8), (7, 9)]
[('delete', 0, -1), ('sub', 1, 0), ('delete', 3, 1)]
{0: ' ', 1: 'S', 3: ' '}
  i    ||     | f | ø | l | g | e    ||  yr  ||   d | o | t    ||  no  ||  kommer  ||  regnet  ||  fra  ||  øst  || 
       ||   i | f | ø | l | g | e    ||  yr  ||     |   |      ||  no  ||  kommer  ||  regnet  ||  fra  ||  øst  || 
  D    ||   I |   |   |   |   |      ||      ||   D | D | D    ||      ||          ||          ||       ||       || 


In [1]:
ref = 'men etter hvert så ble jeg vant til det og ble vant til å være sammen med dyra nedi laben der og det var egentlig ganske trivelig'
hyp = 'men eee etter hvert så ble det vanlig og ble vant til å være samme dyra ned i laben er og det var egentlig ganske trivelig'

# ref = 'men     etter hvert så ble jeg vant   til det og ble vant til å være sammen med dyra nedi  laben der og det var egentlig ganske trivelig'
# hyp = 'men eee etter hvert så ble det vanlig         og ble vant til å være samme      dyra ned i laben er  og det var egentlig ganske trivelig'

from match_em.distances import distance
from match_em.alignments import get_alignment_words, print_alignment_words
from copy import deepcopy

dist3 = distance(ref, hyp)

print([(op if op[0] != 'replace' else ('sub', op[1], op[2])) for op in dist3.get_levenshtein_editops() ])
ct_1, ic_1, ref_1, hyp_1 = get_alignment_words(ref, hyp, dist3.get_levenshtein_editops())
print(ic_1)
word_align = print_alignment_words(ref_1, hyp_1, index_changes=ic_1, only_print_subs=False, do_print=True, print_char_alignments=True, max_width=150)

print()

dist3.compute_weighted_alignment()
candidate_paths = dist3.backtrace_alignment_paths()

print(list(reversed(deepcopy(candidate_paths[0]))))
print(dist3.create_editops(candidate_paths[0]))
ct_2, ic_2, ref_2, hyp_2 = get_alignment_words(ref, hyp, dist3.create_editops(candidate_paths[0]))
print(ic_2)
word_align = print_alignment_words(ref_2, hyp_2, index_changes=ic_2, only_print_subs=False, do_print=True, print_char_alignments=True, max_width=150)

[('insert', 1, 1), ('delete', 5, 6), ('delete', 6, 6), ('sub', 7, 6), ('sub', 8, 7), ('sub', 15, 14), ('sub', 16, 15), ('sub', 17, 16), ('sub', 18, 17), ('sub', 20, 19)]
{1: ' ', 6: ' ', 7: ' ', 8: 'S', 9: 'S', 16: 'S', 17: 'S', 18: 'S', 19: 'S', 21: 'S'}
 men  ||     |   |      ||  etter  ||  hvert  ||  så  ||  ble  ||   j | e | g    ||   v | a | n | t    ||   t | i | l    ||     |   |   | d | e | t   
 men  ||   e | e | e    ||  etter  ||  hvert  ||  så  ||  ble  ||     |   |      ||     |   |   |      ||   d | e | t    ||   v | a | n | l | i | g   
      ||   I | I | I    ||         ||         ||      ||       ||   D | D | D    ||   D | D | D | D    ||   S | S | S    ||   I | I | I | S | S | S   

 ||  og  ||  ble  ||  vant  ||  til  ||  å  ||  være  ||   s | a | m | m | e | n    ||     | m | e | d    ||   d | y | r | a    ||   n | e | d | i    
 ||  og  ||  ble  ||  vant  ||  til  ||  å  ||  være  ||   s | a | m | m | e |      ||   d | y | r | a    ||     | n | e | d    ||     |   

In [2]:
analysis.compute_mistakes(ref, hyp, distance_method='weighted_manual')

NameError: name 'analysis' is not defined

In [7]:
ref = 'men etter hvert så ble jeg vant lig det og ble vant til å være sammen med dyra nedi laben der og det var egentlig ganske trivelig'
# ref = 'men etter hvert så ble jeg vant til det og ble vant til å være sammen med dyra nedi laben der og det var egentlig ganske trivelig'
hyp = 'men eee etter hvert så ble det vanlig og ble vant til å være samme dyra ned i laben er og det var egentlig ganske trivelig'

# ref = 'men     etter hvert så ble jeg vant   til det og ble vant til å være sammen med dyra nedi  laben der og det var egentlig ganske trivelig'
# hyp = 'men eee etter hvert så ble det vanlig         og ble vant til å være samme      dyra ned i laben er  og det var egentlig ganske trivelig'

from match_em.distances import distance
from match_em.alignments import get_alignment_words, print_alignment_words
from match_em.alignments import check_word_compounding
from copy import deepcopy

dist4 = distance(ref, hyp)

dist4.compute_weighted_alignment()
candidate_paths = dist4.backtrace_alignment_paths()
ops4 = dist4.create_editops(candidate_paths[0]) 

print(list(reversed(deepcopy(candidate_paths[0]))))
print(ops4)
ct_4, ic_4, ref_4, hyp_4 = get_alignment_words(ref, hyp, ops4)
print(ic_4)
word_align = print_alignment_words(ref_4, hyp_4, index_changes=ic_4, only_print_subs=False, do_print=True, print_char_alignments=True, max_width=150)

print()

ct_5, ic_5, ref_5, hyp_5, cc_5, cb_5 = check_word_compounding(ct_4, ic_4, ref_4, hyp_4)
print(ic_5)
print("Comp created: ", cc_5)
print("Comp broke: ", cb_5)
word_align = print_alignment_words(ref_5, hyp_5, index_changes=ic_5, only_print_subs=False, do_print=True, print_char_alignments=True)
word_align

[0, (0, 0), (1, 1), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (8, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (15, 17), (16, 18), (17, 19), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27)]
[('insert', 0, 1), ('sub', 5, 6), ('sub', 6, 7), ('delete', 7, 7), ('delete', 8, 7), ('sub', 15, 14), ('delete', 16, 14), ('sub', 18, 16), ('insert', 18, 17), ('sub', 20, 19)]
{1: ' ', 6: 'S', 7: 'S', 8: ' ', 9: ' ', 16: 'S', 17: ' ', 19: 'S', 20: ' ', 22: 'S'}
 men  ||     |   |      ||  etter  ||  hvert  ||  så  ||  ble  ||   j | e | g    ||   v | a | n |   |   | t    ||   l | i | g    ||   d | e | t    || 
 men  ||   e | e | e    ||  etter  ||  hvert  ||  så  ||  ble  ||   d | e | t    ||   v | a | n | l | i | g    ||     |   |      ||     |   |      || 
      ||   I | I | I    ||         ||         ||      ||       ||   S |   | S    ||     |   |   | I | I | S    ||   D | D | D    ||   D | D | D    || 

 og

(' men  ||     |   |      ||  etter  ||  hvert  ||  så  ||  ble  ||   j | e | g    ||   v | a | n | t |   | l | i | g    ||   d | e | t    ||  og  ||  ble  ||  vant  ||  til  ||  å  ||  være  ||   s | a | m | m | e | n    ||   m | e | d    ||  dyra  ||   n | e | d |   | i    ||  laben  ||   d | e | r    ||  og  ||  det  ||  var  ||  egentlig  ||  ganske  ||  trivelig  || ',
 ' men  ||   e | e | e    ||  etter  ||  hvert  ||  så  ||  ble  ||   d | e | t    ||   v | a | n |   |   | l | i | g    ||     |   |      ||  og  ||  ble  ||  vant  ||  til  ||  å  ||  være  ||   s | a | m | m | e |      ||     |   |      ||  dyra  ||   n | e | d |   | i    ||  laben  ||     | e | r    ||  og  ||  det  ||  var  ||  egentlig  ||  ganske  ||  trivelig  || ',
 '      ||   I | I | I    ||         ||         ||      ||       ||   S |   | S    ||     |   |   | D | D |   |   |      ||   D | D | D    ||      ||       ||        ||       ||     ||        ||     |   |   |   |   | D    ||   D | D | D    ||    

In [9]:
analysis.compute_mistakes(ref, hyp, distance_method='weighted_manual')['final_print']

'--- UNK_ID (WER: 29.63, compounds created: 1, compounds broken up: 1)---\n men  ||     |   |      ||  etter  ||  hvert  ||  så  ||  ble  ||   j | e | g    ||   v | a | n | t |   | l | i | g    ||   d | e | t    ||  og  ||  ble  ||  vant  ||  til  ||  å  ||  være  ||   s | a | m | m | e | n    ||   m | e | d    ||  dyra  ||   n | e | d |   | i    ||  laben  ||   d | e | r    ||  og  ||  det  ||  var  ||  egentlig  ||  ganske  ||  trivelig  || \n men  ||   e | e | e    ||  etter  ||  hvert  ||  så  ||  ble  ||   d | e | t    ||   v | a | n |   |   | l | i | g    ||     |   |      ||  og  ||  ble  ||  vant  ||  til  ||  å  ||  være  ||   s | a | m | m | e |      ||     |   |      ||  dyra  ||   n | e | d |   | i    ||  laben  ||     | e | r    ||  og  ||  det  ||  var  ||  egentlig  ||  ganske  ||  trivelig  || \n      ||   I | I | I    ||         ||         ||      ||       ||   S |   | S    ||     |   |   | D | D |   |   |      ||   D | D | D    ||      ||       ||        ||       ||  

In [5]:
ref = 'her i byen har jeg stort sett etter studietilværelsen jobba som lærer'
hyp = 'rybyend har stort sett eee etter studietilværelsen jobba som lærer'

# ref = 'her i  byen    har jeg stort sett     etter studietilværelsen jobba som lærer'
# hyp = '      rybyend  har     stort sett eee etter studietilværelsen jobba som lærer'

from match_em.distances import distance
from match_em.alignments import get_alignment_words, print_alignment_words
from match_em.alignments import check_word_compounding
from copy import deepcopy

dist4 = distance(ref, hyp)

dist4.compute_weighted_alignment()
candidate_paths = dist4.backtrace_alignment_paths()
ops4 = dist4.create_editops(candidate_paths[0]) 

ct_4, ic_4, ref_4, hyp_4 = get_alignment_words(ref, hyp, ops4)

ct_5, ic_5, ref_5, hyp_5, cc_5, cb_5 = check_word_compounding(ct_4, ic_4, ref_4, hyp_4)
word_align = print_alignment_words(ref_5, hyp_5, index_changes=ic_5, only_print_subs=False, do_print=True, print_char_alignments=False, max_width=150)

 her  ||  i  ||   byen    ||  har  ||  jeg  ||  stort  ||  sett  ||       ||  etter  ||  studietilværelsen  ||  jobba  ||  som  ||  lærer  || 
      ||     ||  rybyend  ||  har  ||       ||  stort  ||  sett  ||  eee  ||  etter  ||  studietilværelsen  ||  jobba  ||  som  ||  lærer  || 
  D   ||  D  ||     S     ||       ||   D   ||         ||        ||   I   ||         ||                     ||         ||       ||         || 


In [32]:
from match_em.alignments import check_word_compounding

print(list(reversed(deepcopy(candidate_paths[0]))))
print(dist3.create_editops(candidate_paths[0]))
ct_2, ic_2, ref_2, hyp_2 = get_alignment_words(ref, hyp, dist3.create_editops(candidate_paths[0]))
print(ct_2)
print(ic_2)
check_word_compounding(ct_2, ic_2, ref_2, hyp_2)

[0, (0, 0), (1, 1), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (8, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (15, 17), (16, 18), (17, 19), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27)]
[('insert', 0, 1), ('sub', 5, 6), ('sub', 6, 7), ('delete', 7, 7), ('delete', 8, 7), ('sub', 15, 14), ('delete', 16, 14), ('sub', 18, 16), ('insert', 18, 17), ('sub', 20, 19)]
[(' ', 'eee', 1), ('jeg', 'det', 5), ('vant', 'vanlig', 6), ('til', ' ', 8), ('det', ' ', 9), ('sammen', 'samme', 16), ('med', ' ', 17), ('nedi', 'ned', 19), (' ', 'i', 20), ('der', 'er', 22)]
{1: ' ', 6: 'S', 7: 'S', 8: ' ', 9: ' ', 16: 'S', 17: ' ', 19: 'S', 20: ' ', 22: 'S'}


In [ ]:
ref = 'jeg hadde sett at statoil hadde fått en bot på over hundre millioner kroner'
hyp = 'jeg hadde sett at statoil hadde fått en bot på over hundre millioner kroner'

# ref = 'her i  byen    har jeg stort sett     etter studietilværelsen jobba som lærer'
# hyp = '      rybyend  har     stort sett eee etter studietilværelsen jobba som lærer'

from match_em.distances import distance
from match_em.alignments import get_alignment_words, print_alignment_words
from match_em.alignments import check_word_compounding
from copy import deepcopy

dist4 = distance(ref, hyp)

dist4.compute_weighted_alignment()
candidate_paths = dist4.backtrace_alignment_paths()
ops4 = dist4.create_editops(candidate_paths[0]) 

ct_4, ic_4, ref_4, hyp_4 = get_alignment_words(ref, hyp, ops4)

ct_5, ic_5, ref_5, hyp_5, cc_5, cb_5 = check_word_compounding(ct_4, ic_4, ref_4, hyp_4)
word_align = print_alignment_words(ref_5, hyp_5, index_changes=ic_5, only_print_subs=False, do_print=True, print_char_alignments=False, max_width=150)

## Testing against sample df

In [10]:
from wav2vec_wer import analysis
import pandas as pd

sample_df = pd.read_csv('sample_df.csv')
sample_df.head(3)

references = list(sample_df['transcript_cleaned'])
hypotheses = list(sample_df['wav2vec'])
utt_ids = list(sample_df['utt_id'])

results = analysis.compute_mistakes(
    references, 
    hypotheses, 
    utt_ids, 
    distance_method='weighted_manual', 
    print_to_file='word_level_alignments_compounding_insertdelete_manualWeighted.txt'
)

In [2]:
print(analysis.wer(results))
print(analysis.cer(results))

20.199
5.694


In [3]:
from operator import itemgetter

for k, v in results.items():
    if k == 'final_print':
        pass
    elif k in ['word_miss_pairs', 'compound_miss_pairs', 'char_miss_pairs_word_bound', 'char_miss_pairs_word_unbound', 'known_compounds_achieved', 'known_coupounds_missed']:
        print_list = sorted([(k2, v2) for k2, v2 in results[k].items()], key=itemgetter(1), reverse=True)
        print(k)
        print(print_list)
    else:
        print("{}: {}".format(k, v))


word_level_errors: 122
word_level_references: 604
char_level_errors: 210
char_level_references: 3688
compounds_created: 9
compounds_deleted: 8
word_miss_pairs
[(('frå', 'fra'), 5), (('eit', 'et'), 5), (('høgskulen', 'høgskolen'), 4), (('styre', 'styret'), 3), (('av', ' '), 2), (('vert', 'var'), 2), (('dei', 'de'), 2), (('eg', 'jeg'), 2), (('også', 'og'), 2), (('kalt', 'såkalt'), 2), (('cornflakes', 'kårnfleks'), 2), (('eige', 'eget'), 2), (('zurich', 'syri'), 1), (('reykjavik', 'rekjavik'), 1), (('veke', 'veka'), 1), (('sjølvstendig', 'sjølstendig'), 1), (('eining', 'ening'), 1), (('eige', 'eiet'), 1), (('heimgarden', 'heimgaren'), 1), (('ville', 'villand'), 1), (('han', ' '), 1), (('halda', 'holda'), 1), (('vidare', 'videre'), 1), (('gongene', 'gangene'), 1), (('armane', 'ærmene'), 1), (('hovudet', 'hovedet'), 1), (('sjølv', 'sjøl'), 1), (('leder', 'leider'), 1), (('hanssens', 'hansens'), 1), (('vert', 'vært'), 1), (('hoff', 'hof'), 1), (('aschehoug', 'askehaug'), 1), (('jævla', 'jabl

In [22]:
import match_em.distances as distances

dist = distances.distance('the red cat', 'the rad')
dist.compute_unweighted_alignment()
dist.distance_matrix

array([[0., 1., 2., 3.],
       [1., 0., 1., 2.],
       [2., 1., 1., 2.]])

In [23]:
dist.backtrace_array

[[0, (0, 0), (0, 1), (0, 2)],
 [(0, 0), [(0, 0)], [(1, 1)], [(1, 2)]],
 [(1, 0), [(1, 1)], [(1, 1)], [(2, 2), (1, 2)]]]

In [20]:
from match_em.analysis import compute_mistakes

print(compute_mistakes('the red cat', 'the rad')['final_print'])

--- UNK_ID (WER: 66.67, compounds created: 0, compounds broken up: 0)---
 the  ||   r | e | d    ||   c | a | t    || 
 the  ||     |   |      ||   r | a | d    || 
      ||   D | D | D    ||   S |   | S    || 




In [2]:
from match_em.analysis import compute_mistakes

result = compute_mistakes('the red cat', 'the radcat', distance_method='weighted_manual')
print(result['final_print'])
print(result)

--- UNK_ID (WER: 50.0, compounds created: 1, compounds broken up: 0)---
 the  ||   r | e | d |   | c | a | t    || 
 the  ||   r | a | d |   | c | a | t    || 
      ||     | S |   | D |   |   |      || 


{'word_level_errors': 1, 'word_level_references': 2, 'char_level_errors': 2, 'char_level_references': 11, 'compounds_created': 1, 'compounds_deleted': 0, 'word_miss_pairs': defaultdict(<class 'int'>, {}), 'compound_miss_pairs': defaultdict(<class 'int'>, {('red cat', 'radcat'): 1}), 'char_miss_pairs_word_bound': defaultdict(<class 'int'>, {('e', 'a'): 1}), 'char_miss_pairs_word_unbound': defaultdict(<class 'int'>, {('e', 'a'): 1}), 'known_compounds_achieved': defaultdict(<class 'int'>, {}), 'known_coupounds_missed': defaultdict(<class 'int'>, {}), 'final_print': '--- UNK_ID (WER: 50.0, compounds created: 1, compounds broken up: 0)---\n the  ||   r | e | d |   | c | a | t    || \n the  ||   r | a | d |   | c | a | t    || \n      ||     | S |   | D |   |   |      || \n\n'}


## > 1 sub cost

In [2]:
from wav2vec_wer import analysis
import pandas as pd

sample_df = pd.read_csv('sample_df.csv')
sample_df.head(3)

references = list(sample_df['transcript_cleaned'])
hypotheses = list(sample_df['wav2vec'])
utt_ids = list(sample_df['utt_id'])

results_gt_1 = analysis.compute_mistakes(
    references, 
    hypotheses, 
    utt_ids, 
    distance_method='weighted_manual', 
    print_to_file='word_level_alignments_compounding_insertdelete_manualWeighted_greaterThan1Sub.txt',
    allow_greater_than_1_sub_cost=True
)

## python-Levenshtein check_word_compounding error

In [2]:
from wav2vec_wer import analysis

# ref = 'da ble  det  nemlig   slik  at  de spilte en en  hjemme   første kampen og så ble det to  to   da på ullevål'
# hyp = 'da var      nemisikke              spilt  en ein heime  i første kampen og så ble det to torer    på ulleval'

ref = 'da ble det nemlig slik at de spilte en en hjemme første kampen og så ble det to to da på ullevål'
hyp = 'da var nemisikke spilt en ein heime i første kampen og så ble det to torer på ulleval'

res_1 = analysis.compute_mistakes(ref, hyp, print_to_file='temp_alignment.txt')

In [3]:
from match_em.alignments import *
from match_em.distances import *

ref = 'da ble det nemlig slik at de spilte en en hjemme første kampen og så ble det to to da på ullevål'
hyp = 'da var nemisikke spilt en ein heime i første kampen og så ble det to torer på ulleval'

dist = distance(ref, hyp)
computed_editops = dist.get_levenshtein_editops()
changes_tuples, index_changes, ref, hyp = get_alignment_words(ref, hyp, computed_editops)
word_align = print_alignment_words(ref, hyp, index_changes=index_changes, print_char_alignments=False, do_print=True, max_width=150)
print(changes_tuples)
print(index_changes)
changes_tuples, index_changes, ref, hyp, created_compound, brokeup_compound = check_word_compounding(changes_tuples, index_changes, ref, hyp)

 da  ||  ble  ||  det  ||  nemlig  ||  slik  ||     at      ||   de    ||  spilte  ||  en   ||   en    ||  hjemme  ||  første  ||  kampen  ||  og  || 
 da  ||       ||       ||          ||  var   ||  nemisikke  ||  spilt  ||    en    ||  ein  ||  heime  ||    i     ||  første  ||  kampen  ||  og  || 
     ||   D   ||   D   ||    D     ||   S    ||      S      ||    S    ||    S     ||   S   ||    S    ||    S     ||          ||          ||      || 

 så  ||  ble  ||  det  ||  to  ||  to  ||   da    ||  på  ||  ullevål  ||
 så  ||  ble  ||  det  ||      ||  to  ||  torer  ||  på  ||  ulleval  ||
     ||       ||       ||  D   ||      ||    S    ||      ||     S     ||

[('ble', ' ', 1), ('det', ' ', 2), ('nemlig', ' ', 3), ('slik', 'var', 4), ('at', 'nemisikke', 5), ('de', 'spilt', 6), ('spilte', 'en', 7), ('en', 'ein', 8), ('en', 'heime', 9), ('hjemme', 'i', 10), ('to', ' ', 17), ('da', 'torer', 19), ('ullevål', 'ulleval', 21)]
{1: ' ', 2: ' ', 3: ' ', 4: 'S', 5: 'S', 6: 'S', 7: 'S', 8

In [1]:
from match_em.alignments import *
from match_em.distances import *

ref = 'da ble det nemlig slik at de spilte en en hjemme første kampen og så ble det to to da på ullevål'
hyp = 'da var nemisikke spilt en ein heime i første kampen og så ble det to torer på ulleval'

dist = distance(ref, hyp)
computed_editops = dist.get_weighted_editops()
changes_tuples, index_changes, ref, hyp = get_alignment_words(ref, hyp, computed_editops)
word_align = print_alignment_words(ref, hyp, index_changes=index_changes, print_char_alignments=True, do_print=True, max_width=150)
print(changes_tuples)
print(index_changes)
changes_tuples, index_changes, ref, hyp, created_compound, brokeup_compound = check_word_compounding(changes_tuples, index_changes, ref, hyp)

 da  ||   b | l | e    ||  det  ||   n | e | m |   | l | i | g |   |      ||  slik  ||  at  ||  de  ||   s | p | i | l | t | e    ||  en  ||   e |   |
 da  ||   v | a | r    ||       ||   n | e | m | i | s | i | k | k | e    ||        ||      ||      ||   s | p | i | l | t |      ||  en  ||   e | i |
     ||   S | S | S    ||       ||     |   |   |   | S |   | S |   |      ||        ||      ||      ||     |   |   |   |   |      ||      ||     |   |

 n    ||   h | j | e | m | m | e    ||     ||  første  ||  kampen  ||  og  ||  så  ||  ble  ||  det  ||  to  ||   t | o |   |   |      ||  da  ||  på 
 n    ||   h |   | e | i | m | e    ||  i  ||  første  ||  kampen  ||  og  ||  så  ||  ble  ||  det  ||  to  ||   t | o | r | e | r    ||      ||  på 
      ||     |   |   | S |   |      ||     ||          ||          ||      ||      ||       ||       ||      ||     |   |   |   |      ||      ||     

 ||   u | l | l | e | v | å | l    ||
 ||   u | l | l | e | v | a | l    ||
 ||     |   |   

In [12]:
additional_test_pairs_issue_1 = [(l.split(' , ')[0].strip(), l.split(' , ')[1].strip()) for l in '''
    den ligger det er en av de østfrisiske øyene , det ligger en av de østfrisiske øyene
    da ble det nemlig slik at de spilte en en hjemme første kampen og så ble det to to da på ullevål , da var nemisikke spilt en ein heime i første kampen og så ble det to torer på ulleval
    da gifta vi oss i sjømannskirka i palma palma nova , da gifte vi oss i sjømanskirka i palma palmanova
    som var en veldig sånn klassisk veldig populært glass glass farge fram til cirka atten atten førti da fabrikken gikk inn , så var det veldig sånn klassisk veldig populært glass glass farge fram til cirka atten attenførti da fabrikken gikk inn
    og sånn det er en del kurs og prating om det og og så litt om statistikk som regresjonsanalyse og sånn , og som det er en del kurs og prating om det og også litt om statistikk som regresjonsanalyse og så
    så jeg har tenkt å å gå gjennom det som er presentere hva bagama grammatikkene sier om det og og så kontrastere dette herre med et annet konjunktivspråk nemlig moderne fransk , så jeg har tenkt å gå gjennom det som er presentere hva ga gramatikkene sier om det og også kontrastere dette herre med et annet konjunkturforhold nemlig moderne fransk
    det s som er stort sett temaet der er v å forklare hva språkkompetanse er for noe , det som sto seg et temaet der å forklare hva språkkompetanse er for noe

'''.split('\n') if l.strip() != '']

additional_results = []
for test_pair in additional_test_pairs_issue_1:
    additional_results.append(
        analysis.compute_mistakes(test_pair[0], test_pair[1])
    )

## sound similarities to refine char alignments

In [1]:
# from wav2vec_wer.alignments import *
# from match_em.distances import *
from match_em import distances, alignments

ref = 'perler'
hyp = 'pæler'

dist = distances.distance(ref, hyp)
computed_editops = dist.get_weighted_editops()
changes_tuples, index_changes, ref, hyp = alignments.get_alignment_words(ref, hyp, computed_editops)
word_align = alignments.print_alignment_words(ref, hyp, index_changes=index_changes, print_char_alignments=True, do_print=True, max_width=150)
print(changes_tuples)
print(index_changes)
changes_tuples, index_changes, ref, hyp, created_compound, brokeup_compound = alignments.check_word_compounding(changes_tuples, index_changes, ref, hyp)

  p | e | r | l | e | r    || 
  p | æ |   | l | e | r    || 
    | S |   |   |   |      || 
[('perler', 'pæler', 0)]
{0: 'S'}


In [20]:
norwegian_letters_by_phoetic_traits = {
    'consonants' : {
        # order is:
        # voiced, 
        # class (0 - stop, 1 - fricative, 2 - approx, 3 - trill), 
        # nasal, 
        # place (0 - bilabial, 1 - labio-dental, 2 - alveolar, 3 - retroflex (not actuall used), 4 - palatal, 5 - velar, 6 - uvular, 7 - glottal), 
        # lip rounding
        'b' : (1, 0, 0, 0, 0),
        'c' : [
            (0, 0, 0, 5, 0),
            (0, 2, 0, 2, 0)
        ],
        'd' : (1, 0, 0, 2, 0),
        'f' : (0, 2, 0, 1, 0),
        'g' : (1, 0, 0, 5, 0),
        'h' : (0, 2, 0, 7, 0),
        'j' : (1, 3, 0, 4, 0),
        'k' : (0, 0, 0, 5, 0),
        'l' : (1, 3, 0, 2, 0),
        'm' : (1, 0, 1, 0, 0),
        'n' : (1, 0, 1, 2, 0),
        'p' : (0, 0, 0, 2, 0),
        'r' : [
            (1, 1, 0, 2, 0),
            (1, 1, 0, 6, 0)
        ],
        's' : (0, 2, 0, 2, 0),
        't' : (0, 0, 0, 2, 0),
        'v' : (1, 2, 0, 1, 0),
        'w' : (1, 3, 0, 5, 1),
        'z' : (1, 2, 0, 2, 0)
    },
    'vowels' : {
        # we're only going to use front/backness, hight, and rouding. tense/lax seems to be the domain of short vs long vowels and isn't obvious from orthography
        # order is height (0 being highest, 2 being lowest), backness (0 being back, 2 being front), and roundess (0 unround and 1 round)
        # this height and backness puts 0, 0 at the upper right corner of an IPA vowel chart
        'a' : (2, 0, 0),
        'e' : (1, 2, 0),
        'i' : (0, 2, 0),
        'o' : (0, 0, 1),
        'u' : (0, 1, 1),
        'y' : (0, 2, 1),
        'å' : (1, 0, 1),
        'æ' : (2, 2, 0),
        'ø' : (1, 1, 1)
    }
}


# [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'å', 'æ', 'ø']

In [33]:
import numpy as np

def norwegian_character_distance(v1, v2):
    # just euclidian distance
    if isinstance(v1[0], tuple):
        distances = []
        for v1_tup in v1:
            distances.append(norwegian_character_distance(v1_tup, v2))
        return min(distances)
    elif isinstance(v2[0], tuple):
        distances = []
        for v2_tup in v2:
            distances.append(norwegian_character_distance(v1, v2_tup))
        return min(distances)
    else:
        dist = np.sqrt(
            # NOTE this is only designed to work with V1 and V2 being the same length
            sum([np.square(v1[i] - v2[i]) for i in range(len(v1))])
        )
        return dist

# should be very distant
print(norwegian_character_distance(norwegian_letters_by_phoetic_traits['vowels']['i'], norwegian_letters_by_phoetic_traits['vowels']['a'], 'vowel'))
# should be very similar
print(norwegian_character_distance(norwegian_letters_by_phoetic_traits['vowels']['i'], norwegian_letters_by_phoetic_traits['vowels']['y'], 'vowel'))
# should be the same
print(norwegian_character_distance(norwegian_letters_by_phoetic_traits['consonants']['t'], norwegian_letters_by_phoetic_traits['consonants']['t'], 'consonant'))
# should be very similar
print(norwegian_character_distance(norwegian_letters_by_phoetic_traits['consonants']['d'], norwegian_letters_by_phoetic_traits['consonants']['t'], 'consonant'))
# should be more distant
print(norwegian_character_distance(norwegian_letters_by_phoetic_traits['consonants']['b'], norwegian_letters_by_phoetic_traits['consonants']['g'], 'consonant'))
# should be same
print(norwegian_character_distance(norwegian_letters_by_phoetic_traits['consonants']['s'], norwegian_letters_by_phoetic_traits['consonants']['c'], 'consonant'))
# should be 1.414
print(norwegian_character_distance(norwegian_letters_by_phoetic_traits['consonants']['r'], norwegian_letters_by_phoetic_traits['consonants']['c'], 'consonant'))

2.8284271247461903
1.0
0.0
1.0
5.0
0.0
1.4142135623730951


In [14]:
def character_sub_score(ref_char, hyp_char):
    ref_char_type = 'consonant' if ref_char in norwegian_letters_by_phoetic_traits['consonants'] else 'vowel'
    hyp_char_type = 'consonant' if hyp_char in norwegian_letters_by_phoetic_traits['consonants'] else 'vowel'
    if ref_char_type == hyp_char_type:

(1, 0, 2, 0, 0)

In [24]:
np.exp(1)

2.718281828459045

In [25]:
np.exp(2)

7.38905609893065

In [1]:
def string_insert(string, index, insert_me):
    return string[:index] + insert_me + string[index:]

string_insert('hello', 2, '_')

'he_llo'

In [1]:
from match_em.analysis import compute_mistakes

ref = 'og så ble det to to da på ullevål'
hyp = 'og så ble det to torer på ulleval'
# specifically this is testing python-Levenshtein get_alignment_word() aka Issue 1
results = compute_mistakes(ref, hyp, allow_greater_than_1_sub_cost=True)
print(results['final_print'])

--- UNK_ID (WER: 25.0, compounds created: 1, compounds broken up: 0)---
 og  ||  så  ||  ble  ||  det  ||  to  ||   t | o |   | d | a |      ||  på  ||   u | l | l | e | v | å | l    || 
 og  ||  så  ||  ble  ||  det  ||  to  ||   t | o |   | r | e | r    ||  på  ||   u | l | l | e | v | a | l    || 
     ||      ||       ||       ||      ||     |   |   | S | S |      ||      ||     |   |   |   |   | S |      || 




In [1]:
from match_em.analysis import compute_mistakes

results = compute_mistakes('frå neste veke av vart altså', 'fra neste veka var altså', distance_method='Levenshtein', )
print(results['final_print'])

--- UNK_ID (WER: 60.0, compounds created: 1, compounds broken up: 0)---
  f | r | å    ||  neste  ||   v | e | k | e |   | a | v    ||   v | a | r | t    ||  altså  || 
  f | r | a    ||  neste  ||   v | e | k |   |   | a |      ||   v | a | r |      ||  altså  || 
    |   | S    ||         ||     |   |   | D | D |   | D    ||     |   |   | D    ||         || 




In [3]:
import numpy as np

max_vowel_difference = np.sqrt(np.square(2) + np.square(2) + np.square(2))
print(max_vowel_difference)

3.4641016151377544


In [13]:
import json
from match_em.character_distances import get_norwegian_character_sub_cost
import pandas as pd

with open('/localhome/stipendiater/plparson/match-em/match_em/character_data/character_traits.json', 'r') as open_f:
    nor_chars = json.load(open_f)

actual_distances = {}

for v1 in nor_chars['vowels']:
    for v2 in nor_chars['vowels']:
        key = '{}-{}'.format(v1, v2)
        reverse_key = '{}-{}'.format(v2, v1)
        if key not in actual_distances and reverse_key not in actual_distances:
            actual_distances[key] = get_norwegian_character_sub_cost(v1, v2)

pd.DataFrame([[k, v] for k, v in actual_distances.items()], columns=['letter_pair', 'cost']).sort_values(by='cost')


,letter_pair,cost
0,a-a,0.000000
42,æ-æ,0.000000
39,å-å,0.000000
35,y-y,0.000000
30,u-u,0.000000
24,o-o,0.000000
17,i-i,0.000000
9,e-e,0.000000
44,ø-ø,0.000000
34,u-ø,0.288675
